<a href="https://colab.research.google.com/github/aponamarev/AIND/blob/master/colab_machine_translation_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [0]:
!git clone https://github.com/ap-nlp-research/aind2-nlp-capstone.git

Cloning into 'aind2-nlp-capstone'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 82 (delta 21), reused 31 (delta 11), pack-reused 41
Unpacking objects: 100% (82/82), done.


In [0]:
import os
from tqdm import tqdm, tqdm_notebook
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.contrib.rnn import GRUCell

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13098065592828196108
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12580229728513343825
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13073439521009767319
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14800692839
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6679711592725962550
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [0]:
def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')
# Load English data
english_sentences = load_data('aind2-nlp-capstone/data/small_vocab_en')
# Load French data
french_sentences = load_data('aind2-nlp-capstone/data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [0]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [0]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = keras.preprocessing.text.Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk


# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [0]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sentence) for sentence in x])

    return keras.preprocessing.sequence.pad_sequences(x, maxlen=length, padding='post')

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [0]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [0]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [0]:
def train(train_ops: list, metrics: list, inputs: np.ndarray, targets: np.ndarray, epochs: int, batch_size: int):
    n_samples = len(inputs)
    n_batches = (n_samples + batch_size - 1) // batch_size

    metric_names = [m.op.name for m in metrics]

    with tf.get_default_graph().as_default() as graph:

        input_tensors = tf.get_collection('inputs')[0]
        target_tensors = tf.get_collection('targets')[0]

        # Get a TensorFlow session managed by the supervisor.
        with tf.Session(graph=graph) as sess:
            # Initialize all global variables
            _ = sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])

            for epoc in range(epochs):

                pbar = tqdm(range(n_batches), desc="Epoch {}".format(epoc))

                for it in pbar:
                    start = it * batch_size
                    end = min(n_samples, start + batch_size)

                    x, y = inputs[start:end], targets[start:end]

                    _, metrics_output = sess.run([train_ops, metrics], feed_dict={input_tensors: x, target_tensors: y})

                    pbar.set_postfix(dict([(m, v) for m, v in zip(metric_names, metrics_output)]), refresh=True)

### Model 1: RNN (IMPLEMENTATION)
![RNN](https://github.com/ap-nlp-research/aind2-nlp-capstone/blob/master/images/rnn.png?raw=1)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [0]:
def embed_input(x: tf.Tensor, n_words: int, embedding_size: int) -> tf.Tensor:
    # Create embedding
    emdedded_input = tf.nn.embedding_lookup(params=tf.get_variable(name="embedding", shape=(n_words, embedding_size)),
                                            ids=x)
    return emdedded_input


def create_encoder(emdedded_input: tf.Tensor, num_units: int = 64) -> tf.Tensor:
    """

    :param x: tf.placeholder or data input
    :return:
    """
    with tf.variable_scope("encoder"):
        cell = GRUCell(num_units=num_units)
        _, encoder_final_state = tf.nn.dynamic_rnn(cell=cell, inputs=emdedded_input, dtype=tf.float32)

    return encoder_final_state


def create_decoder(encoder_hs: tf.Tensor, sequence_length: int) -> tf.Tensor:
    batch_size = tf.shape(encoder_hs)[0]
    encoder_units = encoder_hs.get_shape().as_list()[-1]
    dtype = encoder_hs.dtype
    # create a decoder cell

    def teacher_forcing_loop(time, cell_output = None, cell_state = None, loop_state = None):
        emit_output = cell_output  # == None for time == 0
        elements_finished = (time >= sequence_length)
        # time == 0 initialize the sequence with encoder hidden state
        # otherwise, force the cell output as RNNCell input
        if cell_output is None:  # time == 0
            next_cell_state = encoder_hs
            next_input = tf.zeros([batch_size, encoder_units], dtype=dtype)
        else:
            next_cell_state = cell_state
            finished = tf.reduce_all(elements_finished)
            next_input = tf.cond(
                    finished,
                    lambda: tf.zeros([batch_size, encoder_units], dtype=dtype),
                    lambda: cell_output)
        next_loop_state = None
        return (elements_finished, next_input, next_cell_state,
                emit_output, next_loop_state)

    with tf.variable_scope("decoder"):
        cell = GRUCell(num_units=encoder_units)

        # unroll the sequence reusing cell output as the next input
        outputs_ta, decoder_final_state, _ = tf.nn.raw_rnn(cell, loop_fn=teacher_forcing_loop)

        # outputs provided in the form of tensor array that should be converted back into a tensor
        decoder_output = outputs_ta.stack()

    return decoder_output


def create_encoder_decoder_model(inputs: tf.Tensor,
                                 source_size: int,
                                 target_size: int,
                                 target_length: int,
                                 embedding_size: int = 16) -> tf.Tensor:
    embedded_input = embed_input(inputs, n_words=source_size, embedding_size=embedding_size)
    embedded_input = tf.layers.batch_normalization(embedded_input)
    encoder_hs = create_encoder(embedded_input, num_units=256)
    decoder_output = create_decoder(encoder_hs, sequence_length=target_length)
    decoder_output = tf.layers.batch_normalization(decoder_output)
    logits = tf.layers.Dense(units=target_size)(decoder_output)

    return logits

In [0]:
# refresh the graph to make sure nothing was left there from prior runs
tf.reset_default_graph()

inputs = tf.placeholder(tf.int32, [None, max_english_sequence_length], name='inputs')
tf.add_to_collection(name="inputs", value=inputs)
targets = tf.placeholder(tf.int32, [None, max_french_sequence_length], name='targets')
tf.add_to_collection(name="targets", value=targets)

logits = create_encoder_decoder_model(inputs=inputs,
                                      source_size=english_vocab_size+1,
                                      target_size=french_vocab_size+1,
                                      target_length=max_french_sequence_length)
# To be consumed correctly be TF metrics and losses, logits should be transposed [1, 0, 2]
logits = tf.transpose(logits, [1, 0, 2])

# build a loss function
loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels=targets, logits=logits), name='acc_loss')
# build accuracy metric
predictions = tf.argmax(logits, axis=2, name="prediction")
accuracy, update_count_op = tf.metrics.accuracy(labels=targets, predictions=predictions)

variables = tf.trainable_variables()
gradients = tf.gradients(loss, variables)

clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

# Optimization
global_step = tf.train.get_or_create_global_step()
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
update_step = optimizer.apply_gradients(zip(clipped_gradients, variables))

train(train_ops=[update_step, update_count_op, global_step],
      metrics=[loss, accuracy],
      inputs=preproc_english_sentences,
      targets=preproc_french_sentences,
      epochs=20,
      batch_size=1024)

Epoch 19: 100%|██████████| 135/135 [00:08<00:00, 16.79it/s, acc_loss=0.601, accuracy/value=0.709]


## Prediction (IMPLEMENTATION)

In [0]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Pass pretrained model
    model = bd_embedded_rnn_model

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

NameError: ignored

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [0]:
!!python -m nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?